In [9]:
from numba import njit
import numpy as np
from lib.utils import create_windows, file_to_data_frame, ExtendedProtfolio
import pandas as pd
import vectorbt as vbt

In [10]:
# leemos el csv
(s_name, ohlcv) = file_to_data_frame(
    "/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/shorts/Binance_ADAUSDT_minute_3000.csv")
# agarramos solo las columnas que necesitamos
cols = ohlcv.columns
print(cols)
# ohlcv.get(["Open", "High", "Low", "Close", "Volume"]).vbt.ohlcv.plot().show_png()
ohlc = ohlcv.get(["Open", "High", "Low", "Close"])
print(ohlc.head())

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Volume USDT', 'Tradecount'], dtype='object')
                        Open     High      Low    Close
date                                                   
2021-03-16 22:09:00  1.22607  1.22663  1.22074  1.22257
2021-03-16 22:10:00  1.22189  1.22253  1.21917  1.22166
2021-03-16 22:11:00  1.22137  1.22319  1.21804  1.22300
2021-03-16 22:12:00  1.22300  1.22410  1.21878  1.22176
2021-03-16 22:13:00  1.22144  1.22173  1.21876  1.21928


In [11]:
# creamos las ventanas
figure, windows = create_windows(ohlc=ohlc, n=5, window_len=0.6, training_set_len=0.4)
(training_df, train_indexes), (test_df, test_indexes) = windows
# figure.show_png()

In [12]:
portfolio_kwargs = dict(
    direction='all',  # long and short todo falta ver esto
    freq='d'  # todo falta ver esto
)

In [13]:
# simulamos Buy&Hold de cada training y test window y tomamo el expected log returns (elr)
close_columns = list(filter(lambda col: "Close" in col[1], training_df.columns))
in_hold_elr = ExtendedProtfolio.from_holding(training_df[close_columns], **portfolio_kwargs).expected_log_returns()
out_hold_elr = ExtendedProtfolio.from_holding(test_df[close_columns], **portfolio_kwargs).expected_log_returns()
print(in_hold_elr, out_hold_elr)


split_idx       
0          Close    0.000009
1          Close    0.000092
2          Close    0.000091
3          Close    0.000120
4          Close    0.000113
Name: apply_and_reduce, dtype: float64 split_idx       
0          Close    0.000174
1          Close    0.000060
2          Close    0.000026
3          Close   -0.000133
4          Close   -0.000214
Name: apply_and_reduce, dtype: float64


In [14]:
# Creamos el alpha
def alpha_from_df(ohlc_windows):
    alpha = pd.DataFrame()
    for w, new_df in ohlc_windows.groupby(level=0, axis=1):
        alpha[w] = (new_df[w, "Close"] - new_df[w, "Open"]) / (new_df[w, "High"] - new_df[w, "Low"] + 0.001)
    return alpha


In [15]:
# creamos el indicador
@njit
def apply_func_nb(open: np.ndarray, high: np.ndarray, low: np.ndarray, close: np.ndarray, buy_threshold: float,
                  sell_threshold: float):
    aux = (close - open) / (high - low + 0.001)
    aux = np.where(aux >= buy_threshold, 1, aux)
    aux = np.where(aux <= sell_threshold, -1, aux)
    return aux


AlphaInd = vbt.IndicatorFactory(
    input_names=['open', 'high', 'low', 'close'],
    param_names=['buy_threshold', 'sell_threshold'],
    output_names=['signal']
).from_apply_func(apply_func_nb)

# dir(AlphaInd)

def simulate_all_params(ohlc_windows, params_range):
    # creamos las señales
    open, high, low, close = list(map(lambda tu: tu[1].to_numpy(), ohlc_windows.groupby(level=1, axis=1)))
    print("open", open.shape)
    momentum = AlphaInd.run(open=open, high=high, low=low, close=close,
                            buy_threshold=params_range, sell_threshold=params_range,
                            param_product=True,
                            short_name="Momentum")
    ones = np.full(momentum.signal.shape, 1)
    print("ones", ones.shape)
    entry_signal = momentum.signal_equal(ones, crossover=True)
    exit_signal = momentum.signal_equal(-ones, crossover=True)
    print("entry_signal", entry_signal.shape)
    trade_price = ohlc_windows.xs("Close", level=1, axis=1)
    print("trade_price", trade_price.shape)
    return ExtendedProtfolio.from_signals(trade_price, entry_signal, exit_signal, **portfolio_kwargs)


# Simulamos
params_range = np.linspace(0.1, 0.8, 8)
portfolio = simulate_all_params(training_df, params_range)
portfolio.expected_log_returns()

open (1080, 5)
ones (1080, 320)
entry_signal (1080, 320)
trade_price (1080, 5)


ValueError: Index at position 0 could not be aligned